In [1]:
import sys
sys.path.append("/u/dssc/zenocosini/helm_suite/MCQA_Benchmark")
from metrics.utils import *
from metrics.query import DataFrameQuery
from common.tensor_storage import TensorStorage
from common.metadata_db import MetadataDB
from common.utils import *

#from sklearn.feature_selection import mutual_info_regression MISSIN?
from dadapy.data import Data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from sklearn.metrics import pairwise_distances
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
from IPython.display import display
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

from pathlib import Path
import pickle
from collections import Counter

: 

## Functions

In [ ]:
def set_dataframes(db) -> pd.DataFrame:
    """
    Aggregate in a dataframe the hidden states of all instances
    ----------
    hidden_states: pd.DataFrame(num_instances, num_layers, model_dim)
    """
    df = pd.read_sql("SELECT * FROM metadata", db.conn)
    df["train_instances"] = df["train_instances"].astype(str)
    df.drop(columns=["id"],inplace = True)
    #import pdb; pdb.set_trace()
    df.drop_duplicates(subset = ["id_instance"],inplace = True, ignore_index = True) # why there are duplicates???
    return df

In [ ]:
def tensor_retrieve(dict_query):
    query = DataFrameQuery(dict_query)
    hidden_states,logits, hidden_states_df= hidden_states_collapse(metadata_df,tensor_storage,query)
    return hidden_states,logits,hidden_states_df

In [ ]:
tsm =  TensorStorageManager()  

NameError: name 'TensorStorageManager' is not defined

In [ ]:
_PATH = Path("/orfeo/scratch/dssc/zenocosini/mmlu_result/")
result_path = Path(_PATH,"diego")
result_path.mkdir(exist_ok=True,parents=True)
metadata_db = MetadataDB(_PATH / "metadata.db")
metadata_df = set_dataframes(metadata_db)
tensor_storage = TensorStorage(Path(_PATH, "tensor_files"))

# Label distribution

In [9]:
metadata_df["dataset"] = metadata_df["dataset"].apply(lambda x: x.split(":")[1])

In [59]:
rows = []
for dataset in metadata_df["dataset"].unique():
    letter_distribution = metadata_df.query(f"dataset == '{dataset}'")["letter_gold"].value_counts()
    rows.append([dataset,letter_distribution.var(), letter_distribution])
letter_var = pd.DataFrame(rows, columns=["dataset","variance", "letter_distribution"])

In [62]:
letter_var.query("dataset == 'professional_medicine'")["letter_distribution"].values[0]

letter_gold
D    1464
B     660
A     600
C     540
Name: count, dtype: int64

In [61]:
letter_var

,dataset,variance,letter_distribution
0,abstract_algebra,2976.0,letter_gold C 372 B 312 A 264 D 25...
1,anatomy,10116.0,letter_gold C 540 B 408 D 372 A 30...
2,astronomy,21792.0,letter_gold D 612 C 552 B 336 A 32...
3,business_ethics,2208.0,letter_gold A 360 B 312 C 276 D 25...
4,clinical_knowledge,16260.0,letter_gold D 948 B 852 C 696 A 68...
5,college_biology,1056.0,letter_gold D 456 C 444 A 444 B 38...
6,college_chemistry,16608.0,letter_gold D 492 B 252 A 240 C 21...
7,college_computer_science,7968.0,letter_gold D 396 A 312 C 312 B 18...
8,college_mathematics,2688.0,letter_gold D 372 C 300 B 276 A 25...
9,college_medicine,15492.0,letter_gold D 696 C 516 B 432 A 43...
